## ET Prime Data amalgamation

In [1]:
## Libraries

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import date
import datetime as dt

In [2]:
## Inputs

bill_trans=pd.read_csv("01 Prime data\\Billing Transaction(29-Jan).csv")
grp_sub=pd.read_csv("01 Prime data\\Group subscription(29_Jan).csv")
groups1=pd.read_csv("01 Prime data\\Groups(29-Jan).csv")
trans_init=pd.read_csv("01 Prime data\\Transaction Init (29 Jan).csv")
et_pay=pd.read_csv("01 Prime data\\et_pay2.csv")
user_sub=pd.read_csv("01 Prime data\\user_sub.csv")

sub_data=pd.read_csv("01 Prime data\\User Subscriptions Data(29 Jan).csv")

prime_usr_act=pd.read_csv("01 Prime data\\etprimeusers-activity.csv")
article_data=pd.read_csv("01 Prime data\\article_data.csv")

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (12,29,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (43,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
top_art=pd.read_csv("01 Prime data\\art_data_ci.csv",encoding = 'unicode_escape')

#### Transaction init

In [4]:
trans_init=trans_init[(trans_init["plan_id"]==1)|(trans_init["plan_id"]==2)|(trans_init["plan_id"]==3)|(trans_init["plan_id"]==8)]

In [5]:
######### Getting first payment date/ purchase date

# Successful transactiosn filtered
suc_trans=trans_init[
    (trans_init["bill_transaction_payment_state"]=="PAYMENT_SUCCESS")|
    (trans_init["bill_transaction_payment_state"]=="TRANSACTION_SUCCESS")]

# Sorting before removing duplicates
suc1=suc_trans.sort_values(["primary_user_identifier","created_on"])

# Removing duplicates
suc1.drop_duplicates(subset=["primary_user_identifier"],keep='first',inplace=True)

# Renaming columns
suc1.rename(columns={'created_on':'first_purchase_date','user_billing_transaction.final_billing_amount':'first_payment'},inplace=True)

# Keeping only reqd columns
suc2=suc1[["primary_user_identifier","first_purchase_date","first_payment"]]

In [6]:
######### Getting first transaction date (Failure + Success)

# Sorting before removing duplicates
all1=trans_init.sort_values(["primary_user_identifier","created_on"])

# Removing duplicates
all1.drop_duplicates(subset=["primary_user_identifier"],keep='first',inplace=True)

# Renaming columns
all1.rename(columns={'created_on':'first_trans_attempt_date','bill_transaction_payment_state':'first_trans_status'},inplace=True)

# Keeping only reqd columns
all2=all1[["primary_user_identifier","first_trans_attempt_date","first_trans_status"]]

In [7]:
# Merging all till now
new_trans1=trans_init.merge(all2,how='left',on='primary_user_identifier').merge(suc2,how='left',on='primary_user_identifier')
new_trans1.head()

,_class,_id,auto_renew,bill_transaction_payment_state,bill_transaction_source,bill_transaction_state,bill_transanction_type,billing_currency,created_on,expiry_date_from_payment,...,user_billing_transaction.plan_price,user_billing_transaction.platform,user_billing_transaction.product_code,user_billing_transaction.renew_subscription_start_date,user_billing_transaction.trial_avalability,user_billing_transaction.user_subscription_account_id,first_trans_attempt_date,first_trans_status,first_purchase_date,first_payment
0,com.til.subscription.usersubscription.dao.mode...,ETPR20000214894508518520,True,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-01-30T03:06:37.000Z,2019-02-14T00:00:00.000Z,...,2499.0,WEB,ETPR,NaN,True,5ca8f578a503a42ca83484b4,2019-01-30T03:06:37.000Z,PAYMENT_SUCCESS,2019-01-30T03:06:37.000Z,1.0
1,com.til.subscription.usersubscription.dao.mode...,ETPR38567493384779371460,True,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,RENEW,INR,2019-02-13T19:35:04.000Z,2020-02-14T00:00:00.000Z,...,2499.0,WEB,ETPR,2019-02-14T00:00:00.000Z,False,5ca8f578a503a42ca83484b4,2019-01-30T03:06:37.000Z,PAYMENT_SUCCESS,2019-01-30T03:06:37.000Z,1.0
2,com.til.subscription.usersubscription.dao.mode...,ETPR28850770936137955148,True,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-04-01T12:12:57.000Z,2019-04-08T00:00:00.000Z,...,999.0,WEB,ETPR,NaN,True,5ca8f578a503a42ca83484b6,2019-04-01T12:12:57.000Z,PAYMENT_SUCCESS,2019-04-01T12:12:57.000Z,1.0
3,com.til.subscription.usersubscription.dao.mode...,ETPR14352330464066219659,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2018-12-31T22:57:17.000Z,2020-01-14T23:01:31.000Z,...,2499.0,WEB,ETPR,NaN,True,5ca8f578a503a42ca83484b9,2018-12-31T22:57:17.000Z,PAYMENT_SUCCESS,2018-12-31T22:57:17.000Z,2499.0
4,com.til.subscription.usersubscription.dao.mode...,ETPR35829538054508648292,True,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-01-30T03:11:51.000Z,2019-02-14T00:00:00.000Z,...,2499.0,WEB,ETPR,NaN,True,5ca8f578a503a42ca83484bc,2019-01-30T03:11:51.000Z,PAYMENT_SUCCESS,2019-01-30T03:11:51.000Z,1.0


In [8]:
# Getting Refund requirement for trial drops

bill_can=bill_trans[bill_trans['bill_transaction_state']=="CANCELLATION_SUCCESS"]
bill1=bill_can[["user_subscription_account_id","bill_transaction_payment_state"]]
bill1.rename(columns={"bill_transaction_payment_state":"cancellation_state",},inplace=True)

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [9]:
# Meging Cancellation and Refund status

new_trans2=new_trans1.merge(bill1,how='left',left_on="user_billing_transaction.user_subscription_account_id",right_on="user_subscription_account_id")

In [10]:
# Free Trial Dropouts andCancellations Flagged

new_trans2.loc[new_trans2["cancellation_state"]=="NO_REFUND_REQUIRED","renewal_not_done"]=1
new_trans2.loc[(new_trans2["cancellation_state"]=="REFUND_SUCCESS")|(new_trans2["cancellation_state"]=="REFUND_INITIATED"),"free_trial_drop_out"]=1

In [11]:
user_sub_email=user_sub[["user_email","user_mobile","user_sso_id"]]

In [12]:
etp2=et_pay[["Merchant Tran-Id", "Payment Method"]]
new_trans2b=new_trans2.merge(user_sub_email,left_on="primary_user_identifier",right_on="user_sso_id")

In [13]:
new_trans3=new_trans2b.merge(etp2,left_on="_id",right_on="Merchant Tran-Id",how="left")

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#### User Subscription data

In [14]:
## Subscription data and merging with Transaction init

trans_email=new_trans3[["user_billing_transaction.user_subscription_account_id","user_email"]]
trans_email.drop_duplicates(subset=["user_billing_transaction.user_subscription_account_id"],inplace=True)

subdata_email=sub_data.merge(trans_email,how='left',on="user_billing_transaction.user_subscription_account_id")

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [15]:
# Conversion of Dates to proper format

#subdata_email.info()
subdata_email["plan_expiration_details.0.subscriptionTrialEndDate"]=pd.to_datetime(subdata_email["plan_expiration_details.0.subscriptionTrialEndDate"])
subdata_email["plan_expiration_details.0.subscriptionTrialEndDate"] = subdata_email["plan_expiration_details.0.subscriptionTrialEndDate"].dt.tz_convert(None)

subdata_email["plan_expiration_details.0.subscriptionEndDate"]=pd.to_datetime(subdata_email["plan_expiration_details.0.subscriptionEndDate"])
subdata_email["plan_expiration_details.0.subscriptionEndDate"] = subdata_email["plan_expiration_details.0.subscriptionEndDate"].dt.tz_convert(None)

subdata_email["plan_expiration_details.0.subscriptionStartDate"]=pd.to_datetime(subdata_email["plan_expiration_details.0.subscriptionStartDate"])
subdata_email["plan_expiration_details.0.subscriptionStartDate"] = subdata_email["plan_expiration_details.0.subscriptionStartDate"].dt.tz_convert(None)

In [16]:
# Getting Today
today = date.today()

In [17]:
subdata_email['trial_diff']=abs(subdata_email["plan_expiration_details.0.subscriptionTrialEndDate"]-subdata_email["plan_expiration_details.0.subscriptionEndDate"])

In [18]:
# status Flags - Expired/Active

subdata_email["Status"]="Expired"
subdata_email.loc[subdata_email['trial_diff'].dt.days < 5,"Status"]="Cancelled within trial"
subdata_email.loc[subdata_email["plan_expiration_details.0.subscriptionEndDate"]>=today,"Status"]="Active"

subdata_email["trial_diff"]=abs(subdata_email["plan_expiration_details.0.subscriptionTrialEndDate"]-
                      subdata_email["plan_expiration_details.0.subscriptionEndDate"])

subdata_email.loc[subdata_email["plan_expiration_details.0.subscriptionTrialEndDate"]>today,"Status"]="Intrial"

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  # Remove the CWD from sys.path while we load stuff.


In [19]:
# Creating column for plan days before expiry

subdata_email.loc[subdata_email["plan_id"]==1,"sub_days"]=34
subdata_email.loc[subdata_email["plan_id"]==2,"sub_days"]=100
subdata_email.loc[subdata_email["plan_id"]==3,"sub_days"]=380
subdata_email.loc[subdata_email["plan_id"]==8,"sub_days"]=760

In [20]:
# Diff b/w start and end
subdata_email["start-end_days"]=abs(subdata_email["plan_expiration_details.0.subscriptionStartDate"]-subdata_email["plan_expiration_details.0.subscriptionEndDate"])

# Renewal Status
subdata_email.loc[subdata_email["start-end_days"].dt.days > subdata_email["sub_days"],"Renewal_Status"]="Up for Renewal"
subdata_email.loc[(~(subdata_email["start-end_days"].dt.days > subdata_email["sub_days"])),"Renewal_Status"]="Not Up for Renewal"
subdata_email.loc[(~(subdata_email["start-end_days"].dt.days > subdata_email["sub_days"]))&(subdata_email["plan_expiration_details.0.subscriptionEndDate"]<today),"Renewal_Status"]="Up for Renewal"
subdata_email.loc[subdata_email["Status"]=="Cancelled within trial","Renewal_Status"]="Cancelled within trial"

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  import sys


In [21]:
subdata_email["Status"].value_counts()
#.groupby("Renewal_Status")["Email"]

Active                    13686
Expired                   11679
Cancelled within trial     7112
Intrial                      79
Name: Status, dtype: int64

In [22]:
xy=new_trans3.sort_values(by=["user_email","created_on"],ascending=[True,False])
xy.drop_duplicates(subset=["user_email"],keep='first',inplace=True)
autorn=xy[["user_email","auto_renew"]]
subdata_email.drop(columns={"auto_renew"},inplace=True)
sd=subdata_email.merge(autorn,on="user_email",how="left")

In [23]:
subdata_email=sd
#sd[sd["user_email"]=="amit.lakhotia@gmail.com"]

In [24]:
subdata_email.to_csv("subdata_email.csv")

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

In [25]:
renewals=new_trans3[((new_trans3["plan_id"]==1)|(new_trans3["plan_id"]==2)|(new_trans3["plan_id"]==3)|(new_trans3["plan_id"]==8))&
                    (new_trans3["user_billing_transaction.final_billing_amount"]>=399)&
           ((new_trans3["bill_transaction_payment_state"]=="PAYMENT_SUCCESS")|(new_trans3["bill_transaction_payment_state"]=="TRANSACTION_SUCCESS"))]


In [26]:
ren2=pd.DataFrame(renewals.groupby("user_email")["user_billing_transaction.final_billing_amount"].count()).reset_index()

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#### Article Date Creation

In [27]:
## Article Data

#prime_usr_act.head()
#article_data

prime_usr_act["flag_article"]=1
prime_usr_act.loc[prime_usr_act["Articlle"]==0,"flag_article"]=0

In [28]:
art1=prime_usr_act.merge(article_data,left_on="Articlle",right_on="MSID",how="left")

In [29]:
user2=user_sub[["account_status","user_email","user_sso_id"]]
new_trans4=new_trans2b.merge(user2,left_on="primary_user_identifier",right_on="user_sso_id",how="left")

In [30]:
## User - Article grouping

user_art_cnt=pd.DataFrame(art1.groupby(["Email","Platform"])["flag_article"].sum())
user_art_cnt.reset_index(inplace=True)
#user_art_cnt.pivot(index ='user_email', columns ='Platform', values =['flag_article']) 

In [31]:
u1=user_art_cnt[user_art_cnt["Platform"]==11]
u1.rename(columns={"flag_article":"11"},inplace=True)
u2=user_art_cnt[user_art_cnt["Platform"]==12]
u2.rename(columns={"flag_article":"12"},inplace=True)
u3=user_art_cnt[user_art_cnt["Platform"]==19]
u3.rename(columns={"flag_article":"19"},inplace=True)

u1=u1[["Email","11"]]
u2=u2[["Email","12"]]
u3=u3[["Email","19"]]

ux=u1.merge(u2,on="Email").merge(u3,on="Email")

In [32]:
ux["tot"]=ux["11"]+ux["12"]+ux["19"]

In [33]:
ux["11%"]=ux["11"]/ux["tot"]
ux["12%"]=ux["12"]/ux["tot"]
ux["19%"]=ux["19"]/ux["tot"]

In [34]:
ux.loc[ux["19%"]>0.6,"udev_type"]="Primarily App User"
ux.loc[ux["12%"]>0.6,"udev_type"]="Primarily Wap User"
ux.loc[ux["11%"]>0.6,"udev_type"]="Primarily Web User"

ux.loc[((ux["11%"]>0.2)&(ux["12%"]>0.2)),"udev_type"]="Web+Wap User"
ux.loc[((ux["11%"]>0.2)&(ux["19%"]>0.2)),"udev_type"]="Web+App User"
ux.loc[((ux["19%"]>0.2)&(ux["12%"]>0.2)),"udev_type"]="Wap+App User"

ux.loc[((ux["19%"]>0.2)&(ux["12%"]>0.2)&(ux["11%"]>0.2)),"udev_type"]="Wap+Web+App User"

In [35]:
#subdata_email2=subdata_email.merge(art1,left_on="user_email",right_on="Email",how="left")
subdata_artplatform=subdata_email.merge(ux,left_on="user_email",right_on="Email",how="left")
user_art_cnt.rename(columns={"flag_article":"article_count"},inplace=True)
user_artcount=user_art_cnt[["Email","article_count"]]

In [37]:
#subdata_final.merge(ren2,on="user_email",how="left")
ren2.rename(columns={"user_billing_transaction.final_billing_amount":"renewals_done"},inplace=True)

In [38]:
## First level final User Subscription data

subdata_final=subdata_artplatform.merge(ren2,on="user_email",how="left")

In [39]:
## De-duping duplicates on email level

subdata_final.sort_values(by=["user_email","created_on"],ascending=[True,False],inplace=True)
sub_nondup=subdata_final.drop_duplicates(subset=["user_email"],keep="first")
#subdropdup1=subdata2.drop_duplicates(subset=["user_email","plan_id"])

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#### User Level date and Month over month retention - Renewals check

In [40]:
new_trans3["bill_transaction_payment_state"].unique()

array(['PAYMENT_SUCCESS', 'PAYMENT_INTIALIZED', 'PAYMENT_FAILED',
       'PAYMENT_NOT_INTIALIZED', 'TRANSACTION_SUCCESS'], dtype=object)

In [41]:
## Getting Platform Plan-ID, Auto-Renew

suc_init=new_trans3[(new_trans3["bill_transaction_payment_state"]=="PAYMENT_SUCCESS")|
           (new_trans3["bill_transaction_payment_state"]=="TRANSACTION_SUCCESS")]
suc_init.sort_values(by=["plan_id","user_email","created_on"],ascending=[True,True,True],inplace=True)
init_aut_plt_plan=suc_init.drop_duplicates(subset=["user_email"],keep="first")
init_aut_plt_plan=init_aut_plt_plan[["user_email","plan_id","platform","created_on"]]
init_aut_plt_plan.rename(columns={"plan_id":"first_plan_id","platform":"first_platform","created_on":"first_month"},inplace=True)

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [42]:
## Sorting data and getting all date entries

suc_init.sort_values(by=["user_email","created_on"],ascending=[True,False],inplace=True)
suc_init2=suc_init.merge(init_aut_plt_plan,on="user_email",how="left")
suc_init2.loc[suc_init2["plan_id"]==suc_init2["first_plan_id"],"same_plan"]=1
suc_init3=suc_init2[suc_init2["same_plan"]==1]

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
# Getting user level data for current status and removing cancelled within trial users

subdata_email.sort_values(by=["user_email","created_on"],ascending=[True,False],inplace=True)
subdata_email2=subdata_email.drop_duplicates(subset=["user_email"],keep='first')

In [44]:
suc_init4=suc_init3[(suc_init3["user_billing_transaction.final_billing_amount"]==399)|
          (suc_init3["user_billing_transaction.final_billing_amount"]==999)|
          (suc_init3["user_billing_transaction.final_billing_amount"]==2499)|
          (suc_init3["user_billing_transaction.final_billing_amount"]==3599)]

In [45]:
# Ranking by renewals - Removing buy transactions and first payments

suc_init4["created_on"] = pd.to_datetime(suc_init4["created_on"])

suc_init4["renewal_order"]=suc_init4.groupby("user_email")["created_on"].rank("dense",ascending=True)

subdata_st1=subdata_email2[["user_email","Status"]]

#ren_suc=suc_init3[~(suc_init3["renewal_order"]==1)]
ren_suc2=suc_init4.merge(subdata_st1,on="user_email",how="left")

## Final Renewals Data
ren_suc3=ren_suc2[~(ren_suc2["Status"]=="Cancelled within trial")]

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [46]:
suc_init3.loc[(suc_init3["user_email"]=="1vibhor7@gmail.com"),"user_billing_transaction.final_billing_amount"]
#.info()

70    0.0
Name: user_billing_transaction.final_billing_amount, dtype: float64

In [47]:
ren_suc3.to_csv("Renewals.csv")

#-------------------------------------------------------------------------------------------------------------------------------------------------#

#### First 7 and 30 days - Articles Read

In [48]:
art_read=art1
#[art1["Articlle"]>0]

In [49]:
subdata_email.sort_values(by=["user_email","created_on"],ascending=[True,False],inplace=True)
subdata_email2=subdata_email.drop_duplicates(subset=["user_email"],keep='first')

In [81]:
sub2=subdata_email2.merge(art_read,left_on="user_email",right_on="Email",how="left")
sub3=sub2[(sub2["plan_id"]==1)|(sub2["plan_id"]==2)|(sub2["plan_id"]==3)|(sub2["plan_id"]==8)]

In [82]:
#sub3.info()
sub3["created_on"]=pd.to_datetime(sub3["created_on"]).dt.tz_convert(None)
#sub3["created_on"] = sub3["created_on"].dt.tz_convert(None)

sub3["plan_expiration_details.0.subscriptionEndDate"]=sub3["plan_expiration_details.0.subscriptionEndDate"].dt.tz_localize(None)
#sub3["plan_expiration_details.0.subscriptionEndDate"] = sub3["plan_expiration_details.0.subscriptionEndDate"].dt.tz_localize

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [83]:
aug1=pd.to_datetime("2019-08-01 07:21:01.088000+00:00")
sub3["aug1"]=aug1
sub3["aug1"]=sub3["aug1"].dt.tz_convert(None)

jan28=pd.to_datetime("2020-01-28 07:21:01.088000+00:00")
sub3["jan28"]=jan28
sub3["jan28"]=sub3["jan28"].dt.tz_convert(None)

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [84]:
sub3["active_start"]=sub3["aug1"]-sub3["created_on"]
sub3["act_st"]=sub3["created_on"]
sub3.loc[sub3["active_start"].dt.days>0,"act_st"]=sub3["aug1"]

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [85]:
sub3["plan_expiration_details.0.subscriptionEndDate"]=pd.to_datetime(sub3["plan_expiration_details.0.subscriptionEndDate"])

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [86]:
sub3["active_end"]=sub3["jan28"]-sub3["plan_expiration_details.0.subscriptionEndDate"]
sub3["act_end"]=sub3["jan28"]
sub3.loc[sub3["active_end"].dt.days>=0,"act_end"]=sub3["plan_expiration_details.0.subscriptionEndDate"]

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [87]:
sub3["+7_day"]=sub3["act_st"]+pd.DateOffset(days=7)
sub3["-7_day"]=sub3["act_end"]-pd.DateOffset(days=7)

sub3["+30_day"]=sub3["act_st"]+pd.DateOffset(days=30)
sub3["-30_day"]=sub3["act_end"]-pd.DateOffset(days=30)

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [88]:
sub3["Date.1"]=pd.to_datetime(sub3["Date.1"]).dt.tz_localize(None)

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
sub3["plan_id"].unique()

array([2., 8., 3., 1.])

In [89]:
sub3.loc[sub3["plan_id"]==1,"plan_days"]=30
sub3.loc[sub3["plan_id"]==2,"plan_days"]=90
sub3.loc[sub3["plan_id"]==3,"plan_days"]=365
sub3.loc[sub3["plan_id"]==8,"plan_days"]=730

In [90]:
sub3["plan_days_2"]=pd.to_timedelta(sub3["plan_days"], unit='D')

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [91]:
sub3["last_ren_date"]=sub3["act_end"]
sub3.loc[(sub3["Status"]=="Active")&(sub3["Renewal_Status"]=="Up for Renewal"),"last_ren_date"]=sub3["plan_expiration_details.0.subscriptionEndDate"]-sub3["plan_days_2"]
sub3.loc[(sub3["Status"]=="Expired")&(sub3["act_end"]<=sub3["jan28"]),"last_ren_date"]=sub3["plan_expiration_details.0.subscriptionEndDate"]

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
sub3.loc[sub3["created_on"]>=sub3["aug1"],"start_post_aug"]=1

## Within First 7 days - if Date1 is b/w act_st and 7+
## Within last 7 days - if Date1 is b/w act_end and 7-

sub3.loc[((sub3["+7_day"]>=sub3["Date.1"])&(sub3["act_st"]<=sub3["Date.1"])),"article_read_durn1"]="Within first 7"
sub3.loc[((sub3["-7_day"]<=sub3["Date.1"])&(sub3["last_ren_date"]>=sub3["Date.1"])),"article_read_durn1"]="Within last 7"

sub3.loc[((sub3["+30_day"]>=sub3["Date.1"])&(sub3["act_st"]<=sub3["Date.1"])),"article_read_durn2"]="Within first 30"
sub3.loc[((sub3["-30_day"]<=sub3["Date.1"])&(sub3["last_ren_date"]>=sub3["Date.1"])),"article_read_durn2"]="Within last 30"


In [93]:
sub_7day=pd.DataFrame(sub3.groupby(["user_email","Renewal_Status","Status","start_post_aug"])["article_read_durn1"].value_counts())
sub_7day.rename(columns={"article_read_durn1":"article_count"},inplace=True)
sub_7day=sub_7day.reset_index()
                                              
sub_30day=pd.DataFrame(sub3.groupby(["user_email","Renewal_Status","Status","start_post_aug"])["article_read_durn2"].value_counts())
sub_30day.rename(columns={"article_read_durn2":"article_count"},inplace=True)
sub_30day=sub_30day.reset_index()

In [94]:
# No. of days he came
# No. of articles he read

sub4=sub3[(sub3["Date.1"]>=sub3["act_st"])&(sub3["Date.1"]<=sub3["act_end"])]

visits=pd.DataFrame(sub4.groupby("user_email")["Date.1"].nunique()).reset_index()
visits.rename(columns={"Date.1":"visits"},inplace=True)

articles=pd.DataFrame(sub4.groupby("user_email")["Articlle"].nunique()).reset_index()
articles.rename(columns={"Articlle":"articles"},inplace=True)

vis_art=visits.merge(articles,on="user_email",how="outer")

In [95]:
sub4["act_prd"]=sub4['act_end']-sub4['act_st']
sub4["act_prd"]=sub4["act_prd"].dt.days

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
sub5=sub4[["user_email","act_prd"]]
sub5.drop_duplicates(subset=["user_email"],inplace=True)
vis_art2=vis_art.merge(sub5,on="user_email",how="left")

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [97]:
n4=new_trans3[["user_email","platform","auto_renew","plan_id"]]
n4.drop_duplicates(subset=["user_email"],inplace=True)
vis_art3=vis_art2.merge(n4,on="user_email",how="left")

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [98]:
vis_art3.to_csv("visits_articles.csv")

In [99]:
sub_30day.to_csv("first_last_30days.csv")
sub_7day.to_csv("first_last_7days.csv")

###--------------------------------------------------------------------------------------------------------------------------------

#### CPI merging

In [104]:
sub6=sub3

In [105]:
sub6["Articlle"].unique()
sub6.dropna(subset=['Articlle'],inplace=True)
sub6=sub6[sub6["Articlle"]!=0]

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [109]:
sub6["Articlle"]=sub6["Articlle"].astype(int)
top_art["MSID"]=top_art["MSID"].astype(int)

In [111]:
subx=sub6.merge(top_art,left_on="Articlle",right_on="MSID",how="left")

In [116]:
cpi_user=subx.groupby("user_email")["CPI","Exposure CPI","Engagement CPI","Loyalty CPI"].mean()

In [118]:
user_vis_cpi=vis_art3.merge(cpi_user,on="user_email",how="left")

In [120]:
## Users + CPI
user_vis_cpi.to_csv("users_cpi_visits.csv",index=False)

##############-------------------------------------------------------------

#### For Getting the number of articles read before conversions ######

In [3]:
user_email=user_sub[["user_sso_id","user_email","_id"]]
prime_usr_email=prime_usr_act.merge(user_email,left_on="Email",right_on="user_email",how="left")

In [6]:
#prime_usr_email[prime_usr_email["Email"]=="01chinu@gmail.com"]
prime_usr_email.drop(["s","Date.1"],axis=1,inplace=True)

,Date,userid,Email,Platform,Articlle,Type,ID,Section,Story Title,user_sso_id,user_email,_id
18939,10/24/2019 18:52,1.830000e+15,01chinu@gmail.com,11,0,settings,account,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
24492,12/2/2019 11:52,1.830000e+15,01chinu@gmail.com,11,0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
24493,11/27/2019 16:54,1.830000e+15,01chinu@gmail.com,11,0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
24494,11/27/2019 17:18,1.830000e+15,01chinu@gmail.com,11,0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
24495,11/27/2019 17:56,1.830000e+15,01chinu@gmail.com,11,0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
24496,11/18/2019 14:55,1.830000e+15,01chinu@gmail.com,11,0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
24497,11/27/2019 16:43,1.830000e+15,01chinu@gmail.com,11,0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
24498,12/16/2019 12:46,1.830000e+15,01chinu@gmail.com,11,0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
24499,10/25/2019 11:28,1.830000e+15,01chinu@gmail.com,11,0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
24500,11/27/2019 17:37,1.830000e+15,01chinu@gmail.com,11,0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e


In [7]:
suc_init2=trans_init[(trans_init["bill_transaction_payment_state"]=="PAYMENT_SUCCESS")|
           (trans_init["bill_transaction_payment_state"]=="TRANSACTION_SUCCESS")]
suc_init3=suc_init2[(suc_init2["plan_id"]==1)|(suc_init2["plan_id"]==2)|(suc_init2["plan_id"]==3)|(suc_init2["plan_id"]==8)]
suc_init3.sort_values(by=["primary_user_identifier","user_billing_transaction.user_subscription_account_id","created_on"],inplace=True)
suc_init3.drop_duplicates(subset=["user_billing_transaction.user_subscription_account_id"],inplace=True)

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [9]:
suc_init3[suc_init3["user_billing_transaction.user_subscription_account_id"]=="5db1a55584802e561edf549e"]

,_class,_id,auto_renew,bill_transaction_payment_state,bill_transaction_source,bill_transaction_state,bill_transanction_type,billing_currency,created_on,expiry_date_from_payment,...,user_billing_transaction.payment_success_date,user_billing_transaction.plan_code,user_billing_transaction.plan_id,user_billing_transaction.plan_name,user_billing_transaction.plan_price,user_billing_transaction.platform,user_billing_transaction.product_code,user_billing_transaction.renew_subscription_start_date,user_billing_transaction.trial_avalability,user_billing_transaction.user_subscription_account_id
58144,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24T13:20:14.658Z,2021-11-07T13:21:25.948Z,...,2019-10-24T13:21:25.948Z,etp2YearlyPlan,8.0,2-Year Membership,3599.0,WEB,ETPR,NaN,True,5db1a55584802e561edf549e


In [10]:
#suc_init4=suc_init3[["primary_user_identifier","created_on"]]
suc_init5=suc_init3.merge(prime_usr_email,left_on="user_billing_transaction.user_subscription_account_id",right_on="_id",how='left')
#prime_usr2=prime_usr_email.merge(suc_init4,left_on="user_sso_id",right_on="primary_user_identifier",how="left")

In [11]:
suc_init5[suc_init5["user_billing_transaction.user_subscription_account_id"]=="5db1a55584802e561edf549e"]
#suc_init5["created_on"].nunique()

,_class,_id_x,auto_renew,bill_transaction_payment_state,bill_transaction_source,bill_transaction_state,bill_transanction_type,billing_currency,created_on,expiry_date_from_payment,...,Email,Platform,Articlle,Type,ID,Section,Story Title,user_sso_id,user_email,_id_y
97004,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24T13:20:14.658Z,2021-11-07T13:21:25.948Z,...,01chinu@gmail.com,11.0,0.0,settings,account,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
97005,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24T13:20:14.658Z,2021-11-07T13:21:25.948Z,...,01chinu@gmail.com,11.0,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
97006,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24T13:20:14.658Z,2021-11-07T13:21:25.948Z,...,01chinu@gmail.com,11.0,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
97007,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24T13:20:14.658Z,2021-11-07T13:21:25.948Z,...,01chinu@gmail.com,11.0,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
97008,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24T13:20:14.658Z,2021-11-07T13:21:25.948Z,...,01chinu@gmail.com,11.0,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
97009,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24T13:20:14.658Z,2021-11-07T13:21:25.948Z,...,01chinu@gmail.com,11.0,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
97010,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24T13:20:14.658Z,2021-11-07T13:21:25.948Z,...,01chinu@gmail.com,11.0,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
97011,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24T13:20:14.658Z,2021-11-07T13:21:25.948Z,...,01chinu@gmail.com,11.0,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
97012,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24T13:20:14.658Z,2021-11-07T13:21:25.948Z,...,01chinu@gmail.com,11.0,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e
97013,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24T13:20:14.658Z,2021-11-07T13:21:25.948Z,...,01chinu@gmail.com,11.0,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e


In [13]:
suc_init5["created_on"]=pd.to_datetime(suc_init5["created_on"])
suc_init5["created_on"] = suc_init5["created_on"].dt.tz_localize(None)

suc_init5["jul1"]="2019-07-01 00:00:00"
suc_init5["jul1"]=pd.to_datetime(suc_init5["jul1"])
suc_init5["jul1"] = suc_init5["jul1"].dt.tz_localize(None)

suc_init5["Date"]=pd.to_datetime(suc_init5["Date"])
suc_init5["Date"] = suc_init5["Date"].dt.tz_localize(None)

In [14]:
prime_usr2=suc_init5[suc_init5["created_on"]>=suc_init5["jul1"]]

In [16]:
prime_usr2.dropna(subset=["created_on"],inplace=True)

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
prime_usr2.loc[prime_usr2["Date"]<=prime_usr2["created_on"],"before_conversion"]=1
prime_usr2.loc[prime_usr2["Date"]>prime_usr2["created_on"],"before_conversion"]=0

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [20]:
prime_usr2[prime_usr2["user_billing_transaction.user_subscription_account_id"]=="5db1a55584802e561edf549e"]

,_class,_id_x,auto_renew,bill_transaction_payment_state,bill_transaction_source,bill_transaction_state,bill_transanction_type,billing_currency,created_on,expiry_date_from_payment,...,Articlle,Type,ID,Section,Story Title,user_sso_id,user_email,_id_y,jul1,before_conversion
97004,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24 13:20:14.658,2021-11-07T13:21:25.948Z,...,0.0,settings,account,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e,2019-07-01,0.0
97005,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24 13:20:14.658,2021-11-07T13:21:25.948Z,...,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e,2019-07-01,0.0
97006,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24 13:20:14.658,2021-11-07T13:21:25.948Z,...,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e,2019-07-01,0.0
97007,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24 13:20:14.658,2021-11-07T13:21:25.948Z,...,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e,2019-07-01,0.0
97008,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24 13:20:14.658,2021-11-07T13:21:25.948Z,...,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e,2019-07-01,0.0
97009,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24 13:20:14.658,2021-11-07T13:21:25.948Z,...,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e,2019-07-01,0.0
97010,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24 13:20:14.658,2021-11-07T13:21:25.948Z,...,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e,2019-07-01,0.0
97011,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24 13:20:14.658,2021-11-07T13:21:25.948Z,...,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e,2019-07-01,0.0
97012,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24 13:20:14.658,2021-11-07T13:21:25.948Z,...,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e,2019-07-01,0.0
97013,com.til.subscription.usersubscription.dao.mode...,5db1a50e824bdcb21f906830,False,PAYMENT_SUCCESS,ETPAY,TRANSACTION_SUCCESS,BUY,INR,2019-10-24 13:20:14.658,2021-11-07T13:21:25.948Z,...,0.0,editions,archive,NaN,NaN,22umpecumi22ilrhblnzvz9pj,01chinu@gmail.com,5db1a55584802e561edf549e,2019-07-01,0.0


In [44]:
prime_usr2.to_csv("prime_usr_test.csv")

In [75]:
prime_usr2["before_conversion"].value_counts()

0.0    319017
1.0     12151
Name: before_conversion, dtype: int64

In [78]:
## QC Check
prime_usr2.sort_values(by=["Email","before_conversion"],ascending=[True,False],inplace=True)
prime_usrx=prime_usr2.drop_duplicates("Email")
prime_usrx.before_conversion.value_counts()

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0.0    13568
1.0     1815
Name: before_conversion, dtype: int64

In [72]:
prime_usr2.dropna(subset=["Articlle"],inplace=True)
prime_usr2["Articlle"]=prime_usr2["Articlle"].astype(int)
prime_usr3=prime_usr2[(prime_usr2["Articlle"]>0)]
articles_read_b4_conv=pd.DataFrame(prime_usr3.groupby("Email")["before_conversion"].sum())
#articles_read_b4_conv=articles_read_b4_conv.reset_index()

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [77]:
articles_read_b4_conv["before_conversion"].value_counts()

0.0      10431
1.0        579
2.0        307
3.0        199
4.0        104
5.0         73
6.0         57
7.0         45
8.0         31
9.0         12
13.0        11
10.0        10
11.0         9
14.0         8
12.0         8
20.0         7
15.0         6
24.0         3
17.0         3
18.0         3
26.0         2
23.0         2
31.0         2
16.0         2
19.0         2
28.0         2
55.0         1
64.0         1
48.0         1
224.0        1
75.0         1
54.0         1
27.0         1
60.0         1
36.0         1
22.0         1
65.0         1
29.0         1
41.0         1
Name: before_conversion, dtype: int64

In [68]:
articles_read_b4_conv.to_csv("articles_readb4conv.csv")

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
